In [31]:
import torch
import torchvision
import torch.nn as nn
import torch.functional as F

In [1]:
# check Pillow version number
import PIL
print('Pillow Version:', PIL.__version__)

Pillow Version: 9.4.0


In [9]:
from PIL import Image

image = Image.open('data/train/dog.10025.jpg')

print(image.format)
print(image.mode)
print(image.size)

image.show()

JPEG
RGB
(499, 375)


In [4]:
import numpy as np

imdata = np.asarray(image)

print(imdata.shape)

image2 = Image.fromarray(imdata)

print(image2.format)
print(image2.mode)
print(image2.size)

(269, 259, 3)
None
RGB
(259, 269)


In [7]:
from PIL import ImageOps

def padding(image, expected_size):
    delta_width = expected_size[0] - image.size[0]
    delta_height = expected_size[1] - image.size[1]

    pad_width = delta_width // 2
    pad_height = delta_height // 2

    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(image, padding)

def resize_with_padding(image, expected_size):
    image.thumbnail((expected_size[0], expected_size[1]))

    delta_width = expected_size[0] - image.size[0]
    delta_height = expected_size[1] - image.size[1]

    pad_width = delta_width // 2
    pad_height = delta_height // 2

    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(image, padding)

In [14]:
from os import listdir
import matplotlib.image
# import matplotlib.image as img

loaded_images = list()
loaded_labels = list()
count = 0
for filename in listdir('data/train/'):
    if filename.find('cat') != -1:
        loaded_labels.append('cat')
    else:
        loaded_labels.append('dog')

    img = Image.open('data/train/' + filename)
    resized_img = resize_with_padding(img, (100, 100))

    img_data = np.asarray(resized_img)
    loaded_images.append(img_data)
    count += 1
    if count % 200 == 0:
        print("> loaded %d images" % (count))
    # print('> loaded %s %s' % (filename, img_data.shape))
print("Finished")

> loaded 200 images
> loaded 400 images
> loaded 600 images
> loaded 800 images
> loaded 1000 images
> loaded 1200 images
> loaded 1400 images
> loaded 1600 images
> loaded 1800 images
> loaded 2000 images
Finished


In [15]:
print(loaded_labels[100])
test_img = Image.fromarray(loaded_images[100])
test_img.show()

cat


## Data Manipulation

In [23]:
data_set = list(zip(loaded_images, loaded_labels))
print(type(data_set))

<class 'list'>


In [44]:
# print(data_set[0])
print(data_set[0][0].shape)
print(type(data_set[0][0]))
print(data_set[0][1])

(100, 100, 3)
<class 'numpy.ndarray'>
cat


In [26]:
import pandas as pd

df = pd.DataFrame(data_set, columns= ['Image', 'Label'])

In [28]:
df.count()

Image    2000
Label    2000
dtype: int64

In [29]:
df['Label'].value_counts()

cat    1001
dog     999
Name: Label, dtype: int64

In [60]:
# print(df['Label'].values)
label_bool = list()
for label in df['Label'].values:
    label_bool.append(0 if label == 'cat' else 1)
df["Encoded Label"] = label_bool
df.drop('Label', axis=1)

df.describe()

,Encoded Label
count,2000.000000
mean,0.499500
std,0.500125
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


### Split Train Set and Test Set

In [61]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=84)
for train_index, test_index in split.split(df, df['Encoded Label']):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

In [63]:
print(train_set['Encoded Label'].value_counts())
# print(test_set['Image'].value_counts())
print(test_set.describe())

0    801
1    799
Name: Encoded Label, dtype: int64
       Encoded Label
count     400.000000
mean        0.500000
std         0.500626
min         0.000000
25%         0.000000
50%         0.500000
75%         1.000000
max         1.000000


In [43]:
from torchvision import transforms

In [51]:
torch_tensor = torch.from_numpy(df['Image'].values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

### Convert DataFrame to PyTorch Dataset

In [52]:
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [64]:
train_dataset = TensorDataset(df['Image'].values, df['Encoded Label'].values)

TypeError: 'int' object is not callable

In [36]:
class CatDogData(Dataset):
    def __init__(self, df, data_colname, label_colname):
        self.X = torch.tensor(df[data_colname])
        self.y = torch.tensor(df[label_colname])
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [40]:
# catdog_train_set = CatDogData(train_set, 'Image', 'Label')
# train_dataloader = DataLoader(
#     dataset=catdog_train_set,
#     batch_size=10,
#     shuffle=True
# )

# catdog_test_set = CatDogData(test_set, 'Image', 'Label')
# test_dataloader = DataLoader(
#     dataset=catdog_test_set,
#     batch_size=10,
#     shuffle=False
# )

# labels = ('cat', 'dog')

KeyError: 3

## Define a CNN

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            # [100, 100, 3]
            nn.Conv2d(
                in_channels=3,
                out_channels=6,
                kernel_size=5
            ),
            # [96, 96, 6]
            nn.MaxPool2d(
                kernel_size=4,
                stride=4
            ),
            # [24, 24, 6]
            nn.Conv2d(
                in_channels=6,
                out_channels=12,
                kernel_size=5
            ),
            # [20, 20, 12]
            nn.MaxPool2d(
                kernel_size=4,
                stride=4
            ),
            # [5, 5, 12]
            nn.Flatten(),
            # [5*5*12, 1]
            nn.Linear(
                in_features=5*5*12, 
                out_features=120
            ),
            # [1, 120]
            nn.ReLU(),
            nn.Linear(
                in_features=120, 
                out_features=30
            ),
            # [1, 30]
            nn.ReLU(),
            nn.Linear(
                in_features=30, 
                out_features=2
            )
            # [1, 2]
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x